In [ ]:
IBM_Key = kSctzVu5LXmSAPddZC****_******** #private key

In [ ]:
!pip install langdetect

     |████████████████████████████████| 983kB 2.8MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=397808eb0070782932716ae5d76e058343678c9b95de80a3fd9d05c096854592
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [ ]:
pip install --upgrade "ibm-watson>=4.5.0"


     |████████████████████████████████| 378kB 2.8MB/s eta 0:00:01
     |████████████████████████████████| 204kB 8.9MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-4.5.0-cp36-none-any.whl size=364301 sha256=42a7004b895fce92fe8f7b168dedbff54e8d40e738829a9f1942e77c13340402
  Stored in directory: /root/.cache/pip/wheels/71/9a/0a/9b3ca8eca69bc5362eb04709a750b30055a9d27818fd0c9494
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.5.1-cp36-none-any.whl size=44491 sha256=82d5513bece380d74c8cc8567dcd49ee14ea7f87ca80bb091872baabd3d8cbef
  Stored in directory: /root/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson ibm-cloud-sdk-core


In [ ]:
!pip install ibm_cloud_sdk_core

In [ ]:
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException
import json
import os
import pandas as pd
from langdetect import detect
import random
import re
import time
from sklearn.model_selection import train_test_split
import ast

# Raw Data Combination & Preprocessing

In [ ]:
# Read raw datas from the raw data file
path = r'D:\Winter 2020\COVID2019\tweet0120_0220\\'
files = os.listdir(path)

In [ ]:
covid_twitter_data = pd.DataFrame()
# Concat the Twitters data into one-table
for file in files:
    data = pd.read_csv(str(path) + file)
    covid_twitter_data = covid_twitter_data.append(data, ignore_index=True)

ValueError: ignored

In [ ]:
os.chdir(r'D:\Winter 2020\COVID2019')

In [ ]:
# Save the dataset
covid_twitter_data.to_json('febbraiosistemato.json',index=False)


In [ ]:
covid_twitter_data=pd.read_csv('ProvaDataset.csv')

In [ ]:
# Clean text 
t = time.process_time()
clean_text = []
non_en = []
error = []
index = 0
for item in covid_twitter_data['full_text']:
    try:
        index = index +1
        # Delete url in text
        item = re.sub('(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]|(\xa0(…)*)))+){1}','',item)
        # Delete @
        item = re.sub('@[a-zA-Z0-9_.-]+','',item)

        try:
            # Detect whether twitter is in English or not 
            if detect(item) == 'en':
                clean_text.append(item)
            else:
                clean_text.append(None)
                non_en.append(index)
        except:
            error.append(index)
            clean_text.append(None)
    except:
        error.append(index)
        clean_text.append(None)

In [ ]:
# Save error tweets 
error_df = pd.DataFrame(error)
non_en_df = pd.DataFrame(non_en)
error_df.to_csv('error_df.csv',index=False)
non_en_df.to_csv('non_en_df.csv',index=False)

In [ ]:
clean_text = pd.DataFrame(clean_text)
clean_text.columns=['clean_text']
data = pd.concat([covid_twitter_data, clean_text], axis=1)
data.to_csv('covid2019_0120_0220_cleans.csv',index=False)
data.head()

,_id,full_text,lang,clean_text
0,5ef0f9e1f806feafcc979227,RT @mynassah: Front-line health workers haven'...,en,RT : Front-line health workers haven't been pa...
1,5ef0f9e1f806feafcc979229,RT @ShahanR: Today I tested positive for COVID...,en,RT : Today I tested positive for COVID-19. I’m...
2,5ef0f9e1f806feafcc97922b,RT @mattbc: Do you think maybe there's a conne...,en,RT : Do you think maybe there's a connection b...
3,5ef0f9e1f806feafcc97922e,RT @LoyalDefender2K: Although it needs to be s...,en,"RT : Although it needs to be said, I’m genuine..."
4,5ef0f9e1f806feafcc979233,RT @irishmissionun: #COVID19 has impacted all ...,en,RT : #COVID19 has impacted all three pillars o...


In [ ]:
# Drop the row when clean_text is none
data_strip_none = data[data.clean_text.notnull()]
data_strip_none = data_strip_none.set_index(pd.Index(range(0,len(data_strip_none))))

In [ ]:
# random select 2500 samples
random.seed(9001)

data_api, data_test = train_test_split(data_strip_none, train_size=100, random_state=0)

# Reset index
data_api = data_api.set_index(pd.Index(range(0,len(data_api))))
data_test = data_test.set_index(pd.Index(range(0,len(data_test))))

# Save the dataset
data_api.to_csv('data_api.csv',index=False)
data_test.to_csv('data_test_0120_0220.csv',index=False)

# IBM Tone Analyzer

In [ ]:
data_api = pd.read_csv('data_api.csv')

In [ ]:
#attenzione a questo campo
data_api_12000 = data_api.iloc[1:200,]

In [ ]:
data_api_12000.head()

,_id,full_text,lang,clean_text
1,5ef0f9e1f806feafcc979334,RT @openletterbot: 📬 I delivered this ✉️ from ...,en,"RT : 📬 I delivered this ✉️ from Sandra, a 🗳 ve..."
2,5ef0f9e1f806feafcc979255,RT @ohyesitseve: 500 Delta employees test posi...,en,RT : 500 Delta employees test positive for cov...
3,5ef0f9e1f806feafcc979401,RT @GovMikeDeWine: #Ohio's #COVID19 data for J...,en,"RT : #Ohio's #COVID19 data for June 22, 2020.\..."
4,5ef0f9e1f806feafcc979338,RT @KushaKapila: A friend's father is Covid po...,en,RT : A friend's father is Covid positive and h...
5,5ef0f9e1f806feafcc9792c0,RT @Taheri_Movement: #Iran: Shocking charges a...,en,RT : #Iran: Shocking charges against jailed wo...


In [ ]:
authenticator = IAMAuthenticator("lL3HidlNJlRkg7zXVK9MWe*******") #Private
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.eu-gb.tone-analyzer.watson.cloud.ibm.com')
tone_analyzer.set_disable_ssl_verification(False)


In [ ]:
#provo altro metodo di autenticazione NON VA BENE
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import BasicAuthenticator

authenticator = BasicAuthenticator('sylar****','****')

tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.eu-gb.tone-analyzer.watson.cloud.ibm.com')

In [ ]:
#tone_analyzer.set_detailed_response(True)
#response = tone_analyzer.methodName(parameters)
## Access response from methodName
#print(json.dumps(response.get_result(), indent=2))
# # Access information in response headers
#print(response.get_headers())
# # Access HTTP response status
#print(response.get_status_code())

In [ ]:
# try:
#     print(tone_analyzer.tone('Trump is a foolish', content_type='text/plain') )
# except ApiException as ex:
#     print ("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [ ]:
tones = []
index = 0
for item in data_api_12000['clean_text']:
    try:
        index = index + 1
        text = item
        tone_analysis = tone_analyzer.tone({'text': text}, content_type='application/json').get_result()
        tones.append(tone_analysis)
    except ApiException as ex:
        tones.append(None)
        print ("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [ ]:
tones_copy = tones.copy()
tones_copy = pd.DataFrame(tones_copy)

In [ ]:
tones_copy.head()

,document_tone,sentences_tone
0,"{'tones': [{'score': 0.512875, 'tone_id': 'sad...",NaN
1,"{'tones': [{'score': 0.783622, 'tone_id': 'joy...",NaN
2,{'tones': []},"[{'sentence_id': 0, 'text': 'RT : #Ohio's #COV..."
3,"{'tones': [{'score': 0.597764, 'tone_id': 'joy...","[{'sentence_id': 0, 'text': 'RT : A friend's f..."
4,"{'tones': [{'score': 0.848841, 'tone_id': 'con...","[{'sentence_id': 0, 'text': 'RT : #Iran: Shock..."


In [ ]:
data_api_12000 = data_api_12000.reset_index()

In [ ]:
data_api_12000 = data_api_12000.drop(['index'], axis=1)

In [ ]:
data_api_12000 = pd.concat([data_api_12000,tones_copy],axis=1)
data_api_12000.to_csv('data_api_12000.csv',index=False)

# Formating the document tones

In [ ]:
data = pd.read_csv('data_api_12000.csv')


,_id,full_text,lang,clean_text,document_tone,sentences_tone
0,5ef0f9e1f806feafcc979334,RT @openletterbot: 📬 I delivered this ✉️ from ...,en,"RT : 📬 I delivered this ✉️ from Sandra, a 🗳 ve...","{'tones': [{'score': 0.512875, 'tone_id': 'sad...",NaN
1,5ef0f9e1f806feafcc979255,RT @ohyesitseve: 500 Delta employees test posi...,en,RT : 500 Delta employees test positive for cov...,"{'tones': [{'score': 0.783622, 'tone_id': 'joy...",NaN
2,5ef0f9e1f806feafcc979401,RT @GovMikeDeWine: #Ohio's #COVID19 data for J...,en,"RT : #Ohio's #COVID19 data for June 22, 2020.\...",{'tones': []},"[{'sentence_id': 0, 'text': ""RT : #Ohio's #COV..."
3,5ef0f9e1f806feafcc979338,RT @KushaKapila: A friend's father is Covid po...,en,RT : A friend's father is Covid positive and h...,"{'tones': [{'score': 0.597764, 'tone_id': 'joy...","[{'sentence_id': 0, 'text': ""RT : A friend's f..."
4,5ef0f9e1f806feafcc9792c0,RT @Taheri_Movement: #Iran: Shocking charges a...,en,RT : #Iran: Shocking charges against jailed wo...,"{'tones': [{'score': 0.848841, 'tone_id': 'con...","[{'sentence_id': 0, 'text': 'RT : #Iran:\xa0Sh..."
...,...,...,...,...,...,...
94,5ef0f9e1f806feafcc979405,A look at respiratory failure in young healthy...,en,A look at respiratory failure in young healthy...,"{'tones': [{'score': 0.589469, 'tone_id': 'joy...","[{'sentence_id': 0, 'text': 'A look at respira..."
95,5ef0f9e1f806feafcc979395,"RT @DrDenaGrayson: ‼️#Florida passes 100,000 #...",en,"RT : ‼️#Florida passes 100,000 #coronavirus ca...",{'tones': []},"[{'sentence_id': 0, 'text': 'RT : ‼️#Florida p..."
96,5ef0f9e1f806feafcc97930c,RT @PakPMO: This year’s challenge is to make s...,en,RT : This year’s challenge is to make sure tha...,"{'tones': [{'score': 0.543112, 'tone_id': 'con...",NaN
97,5ef0f9e1f806feafcc979284,"@KBeds If that were true, why did he make a tr...",en,"If that were true, why did he make a trade de...","{'tones': [{'score': 0.509573, 'tone_id': 'joy...","[{'sentence_id': 0, 'text': 'If that were true..."


In [ ]:
data.head()

,_id,full_text,lang,clean_text,document_tone,sentences_tone
0,5ef0f9e1f806feafcc979334,RT @openletterbot: 📬 I delivered this ✉️ from ...,en,"RT : 📬 I delivered this ✉️ from Sandra, a 🗳 ve...","{'tones': [{'score': 0.512875, 'tone_id': 'sad...",NaN
1,5ef0f9e1f806feafcc979255,RT @ohyesitseve: 500 Delta employees test posi...,en,RT : 500 Delta employees test positive for cov...,"{'tones': [{'score': 0.783622, 'tone_id': 'joy...",NaN
2,5ef0f9e1f806feafcc979401,RT @GovMikeDeWine: #Ohio's #COVID19 data for J...,en,"RT : #Ohio's #COVID19 data for June 22, 2020.\...",{'tones': []},"[{'sentence_id': 0, 'text': ""RT : #Ohio's #COV..."
3,5ef0f9e1f806feafcc979338,RT @KushaKapila: A friend's father is Covid po...,en,RT : A friend's father is Covid positive and h...,"{'tones': [{'score': 0.597764, 'tone_id': 'joy...","[{'sentence_id': 0, 'text': ""RT : A friend's f..."
4,5ef0f9e1f806feafcc9792c0,RT @Taheri_Movement: #Iran: Shocking charges a...,en,RT : #Iran: Shocking charges against jailed wo...,"{'tones': [{'score': 0.848841, 'tone_id': 'con...","[{'sentence_id': 0, 'text': 'RT : #Iran:\xa0Sh..."


In [ ]:
data = data.iloc[:,1:6]

In [ ]:
data['document_tones'] = data.document_tone.apply(lambda x: ast.literal_eval(x)['tones'])
data.drop('document_tone',axis = 1,inplace = True)

In [ ]:
# split sentiments in corresponding columns
for i in range(0, len(data)):
    for j in range(0, len(data.loc[i, 'document_tones'])):
        dic = data.loc[i, 'document_tones'][j]
        source = dic['tone_id']
        data.loc[i, source] = 1
data.head()

,full_text,lang,clean_text,sentences_tone,document_tones,sadness,joy,confident,tentative,anger,analytical,fear
0,RT @openletterbot: 📬 I delivered this ✉️ from ...,en,"RT : 📬 I delivered this ✉️ from Sandra, a 🗳 ve...",NaN,"[{'score': 0.512875, 'tone_id': 'sadness', 'to...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,RT @ohyesitseve: 500 Delta employees test posi...,en,RT : 500 Delta employees test positive for cov...,NaN,"[{'score': 0.783622, 'tone_id': 'joy', 'tone_n...",NaN,1.0,1.0,NaN,NaN,NaN,NaN
2,RT @GovMikeDeWine: #Ohio's #COVID19 data for J...,en,"RT : #Ohio's #COVID19 data for June 22, 2020.\...","[{'sentence_id': 0, 'text': ""RT : #Ohio's #COV...",[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RT @KushaKapila: A friend's father is Covid po...,en,RT : A friend's father is Covid positive and h...,"[{'sentence_id': 0, 'text': ""RT : A friend's f...","[{'score': 0.597764, 'tone_id': 'joy', 'tone_n...",NaN,1.0,1.0,NaN,NaN,NaN,NaN
4,RT @Taheri_Movement: #Iran: Shocking charges a...,en,RT : #Iran: Shocking charges against jailed wo...,"[{'sentence_id': 0, 'text': 'RT : #Iran:\xa0Sh...","[{'score': 0.848841, 'tone_id': 'confident', '...",NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [ ]:
data


,full_text,lang,clean_text,sentences_tone,document_tones,sadness,joy,confident,tentative,anger,analytical,fear
0,RT @openletterbot: 📬 I delivered this ✉️ from ...,en,"RT : 📬 I delivered this ✉️ from Sandra, a 🗳 ve...",NaN,"[{'score': 0.512875, 'tone_id': 'sadness', 'to...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,RT @ohyesitseve: 500 Delta employees test posi...,en,RT : 500 Delta employees test positive for cov...,NaN,"[{'score': 0.783622, 'tone_id': 'joy', 'tone_n...",NaN,1.0,1.0,NaN,NaN,NaN,NaN
2,RT @GovMikeDeWine: #Ohio's #COVID19 data for J...,en,"RT : #Ohio's #COVID19 data for June 22, 2020.\...","[{'sentence_id': 0, 'text': ""RT : #Ohio's #COV...",[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RT @KushaKapila: A friend's father is Covid po...,en,RT : A friend's father is Covid positive and h...,"[{'sentence_id': 0, 'text': ""RT : A friend's f...","[{'score': 0.597764, 'tone_id': 'joy', 'tone_n...",NaN,1.0,1.0,NaN,NaN,NaN,NaN
4,RT @Taheri_Movement: #Iran: Shocking charges a...,en,RT : #Iran: Shocking charges against jailed wo...,"[{'sentence_id': 0, 'text': 'RT : #Iran:\xa0Sh...","[{'score': 0.848841, 'tone_id': 'confident', '...",NaN,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
94,A look at respiratory failure in young healthy...,en,A look at respiratory failure in young healthy...,"[{'sentence_id': 0, 'text': 'A look at respira...","[{'score': 0.589469, 'tone_id': 'joy', 'tone_n...",NaN,1.0,NaN,1.0,NaN,1.0,NaN
95,"RT @DrDenaGrayson: ‼️#Florida passes 100,000 #...",en,"RT : ‼️#Florida passes 100,000 #coronavirus ca...","[{'sentence_id': 0, 'text': 'RT : ‼️#Florida p...",[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,RT @PakPMO: This year’s challenge is to make s...,en,RT : This year’s challenge is to make sure tha...,NaN,"[{'score': 0.543112, 'tone_id': 'confident', '...",NaN,NaN,1.0,NaN,NaN,NaN,NaN
97,"@KBeds If that were true, why did he make a tr...",en,"If that were true, why did he make a trade de...","[{'sentence_id': 0, 'text': 'If that were true...","[{'score': 0.509573, 'tone_id': 'joy', 'tone_n...",NaN,1.0,NaN,1.0,NaN,NaN,NaN


In [ ]:
data[1].clean_text

KeyError: ignored

In [ ]:
data.to_csv('sentiment_12000.csv',index=False)

# Combining all datasets with tones analyzed by IBM

In [ ]:
path = r'D:\Winter 2020\COVID2019\tweet_sentiment\\'
files = os.listdir(path)

In [ ]:
file

['sentiment_12000.csv',
 'sentiment_2000.csv',
 'sentiment_4500.csv',
 'sentiment_7000.csv',
 'sentiment_9500.csv']

In [ ]:
import pandas as pd
data=pd.read_csv('sentiment_12000.csv')


In [ ]:
covid_twitter_data = pd.DataFrame()
# Concat the Twitters data into one-table
for file in files:
    try:
        data = pd.read_csv(str(path) + file)
    except:
        data = pd.read_csv(str(path) + file,encoding='cp1252')
    covid_twitter_data = covid_twitter_data.append(data, ignore_index=True)

In [ ]:
data['joy'].sum()

31.0

In [ ]:
# Save the dataset
covid_twitter_data.to_csv('sentiment12000full.csv',index=False)